In [1]:
import numpy as np
import dask as d
import cv2
import matplotlib.pyplot as plt
import keras_tuner as kt
import json
import multiprocessing
import os

from glob import glob
from tensorflow import data
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import resnet50

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-05-10 15:49:12.029716: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
from fl_tissue_model_tools import data_prep, dev_config, models, defs
import fl_tissue_model_tools.preprocessing as prep

In [3]:
n_cores = multiprocessing.cpu_count()
n_cores

20

In [4]:
dirs = dev_config.get_dev_directories("../../dev_paths.txt")

# Set up model training parameters

In [5]:
with open("../../model_training/invasion_depth_training_values.json", 'r') as fp:
       training_values = json.load(fp)
training_values["rs_seed"] = None if (training_values["rs_seed"] == "None") else training_values["rs_seed"]

In [6]:
training_values

{'batch_size': 32,
 'frozen_epochs': 50,
 'fine_tune_epochs': 50,
 'val_split': 0.2,
 'early_stopping_patience': 25,
 'early_stopping_min_delta': 0.0001,
 'rs_seed': None,
 'resnet_inp_shape': [128, 128, 3],
 'class_labels': {'no_invasion': 0, 'invasion': 1},
 'cls_thresh': 0.5,
 'n_models': 5}

In [7]:
with open("../../model_training/invasion_depth_hp_space.json", 'r') as fp:
       hp_search_space = json.load(fp)

In [8]:
hp_search_space

{'adam_beta_1_range': [0.8, 0.99],
 'adam_beta_2_range': [0.98, 0.999],
 'frozen_lr_range': [0.0001, 0.01],
 'fine_tune_lr_range': [1e-05, 0.001],
 'last_layer_options': ['conv5_block3_out',
  'conv5_block2_out',
  'conv5_block1_out',
  'conv4_block6_out'],
 'num_initial_points': 25,
 'max_opt_trials': 50}

In [9]:
### Data paths ###
root_data_path = f"{dirs.data_dir}/invasion_data/"
model_training_path = f"{dirs.analysis_dir}/resnet50_invasion_model"
project_name = "invasion_hp_trials"
hypermodel_name = "invasion_depth_hypermodel"
hp_search_hp_path = f"{model_training_path}/hyperparameter_search_hps"
hp_search_weights_path = f"{model_training_path}/hyperparameter_search_weights"
best_hp_file = f"{hp_search_hp_path}/best_hyperparams_v1.json"
mcp_best_frozen_weights_file = f"{hp_search_weights_path}/best_frozen_weights.h5"


### General training parameters ###
resnet_inp_shape = tuple(training_values["resnet_inp_shape"])
class_labels = training_values["class_labels"]
# Binary classification -> only need 1 output unit
n_outputs = 1

seed = training_values["rs_seed"]
val_split = training_values["val_split"]
batch_size = training_values["batch_size"]
frozen_epochs = training_values["frozen_epochs"]
fine_tune_epochs = training_values["fine_tune_epochs"]
# frozen_epochs = 5
# fine_tune_epochs = 5


### Early stopping ###
es_criterion = "val_loss"
es_mode = "min"
# Update these depending on seriousness of experiment
es_patience = training_values["early_stopping_patience"]
es_min_delta = training_values["early_stopping_min_delta"]


### Frozen model saving (for transitioning from frozen model to fine-tuned model) ###
mcp_criterion = "val_loss"
mcp_mode = "min"


### Hyperparameter search ###
adam_beta_1_range = tuple(hp_search_space["adam_beta_1_range"])
adam_beta_2_range = tuple(hp_search_space["adam_beta_2_range"])
frozen_lr_range = tuple(hp_search_space["frozen_lr_range"])
fine_tune_lr_range = tuple(hp_search_space["fine_tune_lr_range"])
last_layer_options = hp_search_space["last_layer_options"]
num_initial_points = hp_search_space["num_initial_points"]
max_opt_trials = hp_search_space["max_opt_trials"]
# num_initial_points = 3
# max_opt_trials = 5

In [11]:
data_prep.make_dir(hp_search_hp_path)
data_prep.make_dir(hp_search_weights_path)

# Prep for loading data

In [12]:
rs = np.random.RandomState(seed)

In [13]:
# Training & validation data (drawn from same image set & randomly assigned)
tv_class_paths = {v: glob(f"{root_data_path}/train/{k}/*.tif") for k, v in class_labels.items()}
for k, v in tv_class_paths.items():
    rs.shuffle(v)

In [14]:
train_data_paths, val_data_paths = data_prep.get_train_val_split(tv_class_paths, val_split=val_split)

# Datasets

In [15]:
train_datagen = data_prep.InvasionDataGenerator(
    train_data_paths,
    class_labels,
    batch_size,
    resnet_inp_shape[:2],
    rs,
    class_weights=True,
    shuffle=True,
    augmentation_function=prep.augment_imgs
)

In [16]:
train_datagen.class_counts

{0: 449, 1: 177}

In [17]:
train_datagen.class_weights

{0: 0.6971046770601337, 1: 1.768361581920904}

In [18]:
val_datagen = data_prep.InvasionDataGenerator(
    val_data_paths,
    class_labels,
    batch_size,
    resnet_inp_shape[:2],
    rs,
    class_weights=train_datagen.class_weights,
    shuffle=True,
    augmentation_function=train_datagen.augmentation_function
)

In [19]:
val_datagen.class_counts

{0: 112, 1: 44}

# Build hyper model

In [20]:
hypermodel = models.ResNet50TLHyperModel(
    n_outputs=n_outputs,
    img_shape=resnet_inp_shape,
    loss=BinaryCrossentropy(),
    weighted_metrics=[BinaryAccuracy()],
    name=hypermodel_name,
    output_act="sigmoid",
    adam_beta_1_range=adam_beta_1_range,
    adam_beta_2_range=adam_beta_2_range,
    frozen_lr_range=frozen_lr_range,
    fine_tune_lr_range=fine_tune_lr_range,
    frozen_epochs=frozen_epochs,
    fine_tune_epochs=fine_tune_epochs,
    base_model_name="base_model",
    # EarlyStopping callback parameters
    es_criterion=es_criterion,
    es_mode=es_mode,
    es_patience=es_patience,
    es_min_delta=es_min_delta,
    # Frozen ModelCheckpoint callback parameters
    mcp_criterion=mcp_criterion,
    mcp_mode=mcp_mode,
    mcp_best_frozen_weights_path=mcp_best_frozen_weights_file
)

2022-05-10 15:54:50.794331: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-10 15:54:50.800914: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-10 15:54:51.395250: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: Tesla M60 computeCapability: 5.2
coreClock: 1.1775GHz coreCount: 16 deviceMemorySize: 7.94GiB deviceMemoryBandwidth: 149.31GiB/s
2022-05-10 15:54:51.395345: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-10 15:54:51.461335: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-05-10 15:54:51.461439: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-05-10 

In [21]:
tuner = kt.BayesianOptimization(
    hypermodel=hypermodel,
    objective="val_loss",
    num_initial_points=num_initial_points,
    max_trials=max_opt_trials,
    seed=seed,
    # directory="../../model_training/",
    directory=model_training_path,
    project_name=project_name
)

In [22]:
# Cannot use external callbacks. Callbacks are defined inside the hypermodel's fit function
tuner.search(
    train_datagen,
    validation_data=val_datagen,
    workers=n_cores
)

Trial 50 Complete [00h 08m 23s]
val_loss: 0.21549305319786072

Best val_loss So Far: 0.19290809333324432
Total elapsed time: 06h 44m 13s
INFO:tensorflow:Oracle triggered exit


In [23]:
tuner.results_summary()

Results summary
Results in /nfs/stak/users/cookcar/hpc-share/fogg_lab_analysis/resnet50_invasion_model/invasion_hp_trials
Showing 10 best trials
Trial summary
Hyperparameters:
last_resnet_layer: conv5_block1_out
frozen_lr: 0.00023296415902195254
adam_beta_1: 0.8690806056354333
adam_beta_2: 0.9817045929249536
fine_tune_lr: 4.837200625807212e-05
Score: 0.19290809333324432
Trial summary
Hyperparameters:
last_resnet_layer: conv5_block2_out
frozen_lr: 0.00695461878420833
adam_beta_1: 0.9164616012708694
adam_beta_2: 0.992845865126449
fine_tune_lr: 1.0869955378279433e-05
Score: 0.19577443599700928
Trial summary
Hyperparameters:
last_resnet_layer: conv5_block3_out
frozen_lr: 0.00037464381329162526
adam_beta_1: 0.8352069226681756
adam_beta_2: 0.981236038221177
fine_tune_lr: 1.03699753028074e-05
Score: 0.19703063368797302
Trial summary
Hyperparameters:
last_resnet_layer: conv5_block3_out
frozen_lr: 0.009801011964790135
adam_beta_1: 0.9711093553420982
adam_beta_2: 0.9925358648256517
fine_tune_lr:

In [24]:
best_hp = tuner.get_best_hyperparameters()[0]

In [25]:
best_hp.values

{'last_resnet_layer': 'conv5_block1_out',
 'frozen_lr': 0.00023296415902195254,
 'adam_beta_1': 0.8690806056354333,
 'adam_beta_2': 0.9817045929249536,
 'fine_tune_lr': 4.837200625807212e-05}

In [26]:
with open(best_hp_file, "w") as fp:
    json.dump(best_hp.values, fp, sort_keys=True)